In [1]:
import torch
import nltk
import numpy as np
import pandas as pd
import gzip
import torch
import requests
import json
from nltk.corpus import brown
from torch.utils.data import Dataset, DataLoader
import random
import io
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from collections import Counter
import torch.nn.functional as F
import requests

download and prepare the Brown corpus from NLTK for training a Word2Vec model, which is used for creating embeddings of words.

Download Squad2 dataset, split it into training and testing sets, and save them to files.

In [2]:
# Download SQuAD2 dataset
url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"
response = requests.get(url)

#Save the dataset to a file
dataset_file = "squad2.json"
with open(dataset_file, 'wb') as f:
   f.write(response.content)

#Split dataset into train and test files
train_ratio = 0.8
train_file = "train.json"
test_file = "test.json"

with open(dataset_file, 'r') as f:
    squad_data = json.load(f)
    data = squad_data['data']

    # Shuffle the data
    random.shuffle(data)

    # Split into train and test sets
    split_index = int(len(data) * train_ratio)
    train_data = data[:split_index]
    test_data = data[split_index:]

    # Save train data to file
    train_squad_data = {'data': train_data}
    with open(train_file, 'w') as train_f:
        json.dump(train_squad_data, train_f)

    # Save test data to file
    test_squad_data = {'data': test_data}
    with open(test_file, 'w') as test_f:
        json.dump(test_squad_data, test_f)

In [3]:
#We will use this function on each file in the dataset - test and train
#only the filename is required as the previous function returns the specified file to the same location

def extract_squad_data(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        file_name = json.load(f)

    paragraphs = []
    for article in file_name['data']:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                question = qa['question']
                answers = [answer['text'] for answer in qa['answers']]
                for answer in answers:
                    paragraphs.append((question, answer))

    df = pd.DataFrame(paragraphs, columns=['Question', 'Answer'])
    return df

In [4]:
#convert .json files to dataframes
train_df = extract_squad_data("train.json") 
test_df = extract_squad_data("test.json")


In [5]:


import pandas as pd
import nltk
import string
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")

def clean_text(df):
    cleaned_df = df.copy()
    for column in cleaned_df:
        cleaned_df[column] = cleaned_df[column].apply(lambda sentence: process_sentence(sentence))
    return cleaned_df

def process_sentence(sentence):
    sentence = ''.join([s.lower() for s in sentence if s not in string.punctuation])
    sentence = ' '.join(stemmer.stem(w) for w in sentence.split())
    tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(sentence)

    return ' '.join(tokens) 
# usage:
# df = clean_text(df)


In [6]:
train_clean_df = clean_text(train_df)


In [7]:
print(train_clean_df.head())

                                            Question              Answer
0  what a common abbrevi for the term institut of...                  it
1  in what centuri did the term polytechn first s...        19th centuri
2  what school found in 1794 first use a form of ...  école polytechniqu
3  what languag doe the french word polytechniqu ...               greek
4  what word mean the same thing as institut of t...           polytechn


In [8]:
class Vocab:
    def __init__(self):
        self.word2index = {}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 4: "<UNK>"}
        self.word2count = {}
        self.n_words = 4  # Count <PAD>, <SOS>, <EOS>, <UNK>
    def build_vocab(self, sentences):
        for sentence in sentences:
            for word in self.tokenize(sentence):
                self.add_word(word)
    def tokenize(self, sentence):
        return sentence.strip().split()
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [9]:

""""

    Tokenizes English text from a string into a list of strings (tokens)
 
    return [tok.text for tok in spacy_en.tokenizer(text)]

   """

'"\n\n    Tokenizes English text from a string into a list of strings (tokens)\n \n    return [tok.text for tok in spacy_en.tokenizer(text)]\n\n   '

In [10]:
question_vocab = Vocab()
answer_vocab = Vocab()


questions = train_clean_df['Question']
answers = train_clean_df['Answer']

question_vocab.build_vocab(questions)
answer_vocab.build_vocab(answers)

question_vocab.add_word('<UNK>')
answer_vocab.add_word('<UNK>')



In [11]:
question_vocab_size = len(question_vocab.word2count)
print(question_vocab_size)

25023


In [12]:
answer_vocab_size = len(answer_vocab.word2count)
print(answer_vocab_size)

28571


In [13]:
print(question_vocab.index2word)

{0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 4: 'what', 5: 'a', 6: 'common', 7: 'abbrevi', 8: 'for', 9: 'the', 10: 'term', 11: 'institut', 12: 'of', 13: 'technolog', 14: 'in', 15: 'centuri', 16: 'did', 17: 'polytechn', 18: 'first', 19: 'show', 20: 'up', 21: 'school', 22: 'found', 23: '1794', 24: 'use', 25: 'form', 26: 'word', 27: 'it', 28: 'name', 29: 'languag', 30: 'doe', 31: 'french', 32: 'polytechniqu', 33: 'come', 34: 'from', 35: 'mean', 36: 'same', 37: 'thing', 38: 'as', 39: 'was', 40: 'world', 41: 'war', 42: 'led', 43: 'to', 44: 'increas', 45: 'popular', 46: 'and', 47: 'braunschweig', 48: 'univers', 49: 'year', 50: 'école', 51: 'kind', 52: 'do', 53: 'countri', 54: 'like', 55: 'germani', 56: 'switzerland', 57: 'consid', 58: 'be', 59: 'were', 60: 'uk', 61: 'given', 62: 'design', 63: 'is', 64: 'royal', 65: 'now', 66: 'street', 67: 'westminst', 68: 'on', 69: 'some', 70: 'describ', 71: 'select', 72: 'that', 73: 'focus', 74: 'educ', 75: 'appli', 76: 'scienc', 77: 'engin', 78: 'switch', 79: '19

In [14]:
print(answer_vocab.index2word)

{0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 4: 'it', 5: '19th', 6: 'centuri', 7: 'école', 8: 'polytechniqu', 9: 'greek', 10: 'polytechn', 11: 'the', 12: 'bergschola', 13: 'world', 14: 'war', 15: 'ii', 16: 'collegium', 17: 'carolinum', 18: '1794', 19: 'institut', 20: 'of', 21: 'higher', 22: 'educ', 23: '1992', 24: 'univers', 25: 'westminst', 26: 'regent', 27: 'street', 28: 'technolog', 29: 'colleg', 30: 'advanc', 31: 'system', 32: 'tafe', 33: 'melbourn', 34: '2009', 35: 'seven', 36: 'fachhochschul', 37: 'master', 38: 'degre', 39: 'hogeschool', 40: 'academ', 41: 'four', 42: 'three', 43: 'cambodia', 44: 'affili', 45: 'school', 46: 'quebec', 47: 'public', 48: 'fund', 49: 'canada', 50: 'collabor', 51: 'instituteindustri', 52: 'project', 53: '1895', 54: 'capit', 55: 'half', 56: '1997', 57: 'geophys', 58: 'quito', 59: 'astronom', 60: 'observatori', 61: 'nation', 62: '1873', 63: 'one', 64: 'countri', 65: 'develop', 66: '50', 67: 'de', 68: 'technologi', 69: '180', 70: '160', 71: 'finland', 72: 'ammatt

In [15]:
def pad_or_cut(df, column_names, length):
    pad_symbol = "<PAD>"
    
    for column_name in column_names:
        df[column_name] = df[column_name].apply(lambda x: (x.split()[:length] + [pad_symbol]*length)[:length])
        df[column_name] = df[column_name].apply(lambda x: ' '.join(x))
    return df


In [16]:

padded_df = pad_or_cut(train_clean_df, ['Question', 'Answer'], 10)

In [17]:
def words_to_indices(df, question_vocab, answer_vocab):
    df['Question'] = df['Question'].apply(lambda sentence: [question_vocab.word2index[word] if word in question_vocab.word2index else question_vocab.word2index['<UNK>'] for word in sentence.split()])
    df['Answer'] = df['Answer'].apply(lambda sentence: [answer_vocab.word2index[word] if word in answer_vocab.word2index else answer_vocab.word2index['<UNK>'] for word in sentence.split()])
    return df

In [18]:
idx_df = words_to_indices(padded_df, question_vocab, answer_vocab) # could have used nn.padding & nn.embedding

In [19]:
idx_df.head(20)

from torch.utils.data import TensorDataset, DataLoader

In [20]:
train_src = idx_df['Question'].tolist() #extract index seqences from df
train_trg = idx_df['Answer'].tolist()

In [21]:
QA_pairs = [(src, trg) for src, trg in zip(train_src, train_trg) if len(src) == len(trg)]

In [22]:
train_src = [pair[0] for pair in QA_pairs]
train_trg = [pair[1] for pair in QA_pairs]

In [23]:
source_tensor = torch.tensor(train_src, dtype=torch.long) 
target_tensor = torch.tensor(train_trg, dtype=torch.long)

In [24]:

"""PAD_IDX = vocabulary.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)"""

"PAD_IDX = vocabulary.stoi['<pad>']\ncriterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)"

In [25]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, x, hidden, cell_state):
        x = self.embedding(x)
        x = x.view(len(x), 1, -1)  # Adjust for sequence length
        x, (hidden, cell_state) = self.lstm(x, (hidden, cell_state))
        return x, hidden, cell_state



In [26]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, self.hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden, cell_state):
        x = self.embedding(x)
        x = x.view(len(x), 1, -1)  # Adjust for sequence length
        x, (hidden, cell_state) = self.lstm(x, (hidden, cell_state))
        x = self.softmax(self.fc(x[0]))
        return x, hidden, cell_state

In [27]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.encoder = Encoder(self.input_size, self.hidden_size)
        self.decoder = Decoder(self.hidden_size, self.output_size)
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        src_len = src.shape[0]
        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        trg_vocab_size = self.output_size
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)
        
        encoder_hidden = torch.zeros([1, 1, self.hidden_size]).to(device)
        cell_state = torch.zeros([1, 1, self.hidden_size]).to(device)
        
        for i in range(src_len):
            encoder_output, encoder_hidden, cell_state = self.encoder(src[i], encoder_hidden, cell_state)
            
        decoder_input = torch.Tensor([[0]]).long().to(device)
        decoder_hidden = encoder_hidden
        decoder_cell_state = cell_state

        for t in range(trg_len):
            decoder_output, decoder_hidden, decoder_cell_state = self.decoder(decoder_input, decoder_hidden, decoder_cell_state)
            outputs[t] = decoder_output
            top1 = decoder_output.argmax(1)
            decoder_input = top1 if random.random() < teacher_forcing_ratio else trg[t]
            assert t < len(trg), f"Index out of bounds: Index is {t} but trg length is {len(trg)}"

        
        return outputs
        
    

In [28]:
import torch.nn as nn

In [29]:
import matplotlib.pyplot as plt

In [30]:
question_vocab_size = len(question_vocab.word2count)
answer_vocab_size = len(answer_vocab.word2count)

In [31]:


def create_model(input_size, hidden_size, output_size):
    # Instantiate the Seq2Seq model
    seq2seq = Seq2Seq(input_size, hidden_size, output_size).to(device)

    return seq2seq


In [32]:
from torch.utils.data import TensorDataset, DataLoader

In [33]:
print("Input Tensor Shape:", source_tensor.shape)
print("Target Tensor Shape:", target_tensor.shape)

Input Tensor Shape: torch.Size([69405, 10])
Target Tensor Shape: torch.Size([69405, 10])


In [34]:
dataset = TensorDataset(source_tensor, target_tensor)
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [44]:
def train_model(seq2seq, dataloader, optimizer, criterion, device):
    seq2seq.train()  # Ensure the network is in train mode
    total_loss = 0

    for source_tensor, target_tensor in dataloader:
        # Send tensors to the correct device
        source_tensor = source_tensor.to(device)
        target_tensor = target_tensor.to(device)

        # Transpose the tensors to [seq_len, batch_size]
        source_tensor = source_tensor.transpose(0, 1)
        target_tensor = target_tensor.transpose(0, 1)

        optimizer.zero_grad()  # Reset the gradients

        output = seq2seq(source_tensor, target_tensor)

        # Adjust the shapes of output and target_tensor for the loss function
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        target_tensor = target_tensor[1:].view(-1)

        loss = criterion(output, target_tensor)  # Compute the loss

        loss.backward()  # Perform backpropagation

        torch.nn.utils.clip_grad_norm_(seq2seq.parameters(), 1)  # Clip the gradients to prevent explosion

        optimizer.step()  # Update the weights

        total_loss += loss.item()  # Accumulate the loss

    return total_loss / len(dataloader)  # Return the average loss




In [45]:
input_size = question_vocab_size
hidden_size = 128
output_size = answer_vocab_size
num_epochs = 10

In [37]:
device = torch.device("cpu")

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [47]:
device


device(type='cuda')

In [48]:
#Hyperparameters

learning_rate = 0.001
num_epochs = 10
optimizer = torch.optim.SGD(seq2seq.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [49]:
training_losses = []
for epoch in range(num_epochs):
    loss = train_model(seq2seq, dataloader, optimizer, criterion, device)
    training_losses.append(loss)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}')

# Plot the training progress
plt.figure()
plt.plot(training_losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()

../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1093: indexSelectSmallIndex: block: [0,0,0], thread: [6,0,0

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def QAPairs(df):
    temp1 = df['Question'].apply(lambda x: " ".join(x) ).to_list()
    temp2 = df['Answer'].apply(lambda x: " ".join(x) ).to_list()
    return [list(i) for i in zip(temp1, temp2)]   